In [1]:
"""Libraries"""

# for plotting:
%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 18})

import numpy as np   # matrices, math
import os    # file management
import pandas as pd   # data frames
import importlib   # for reloading your own files

# my own file:
import full_bubble_model as de    # full bubble model
importlib.reload(de)   # reload changes you made

model: chem_Otomo2018_without_O_reactions
Enable heat transfer: True, enable evaporation: False, enable reactions: True
model: chem_Otomo2018_without_O_reactions
Enable heat transfer: True, enable evaporation: False, enable reactions: True


<module 'full_bubble_model' from 'C:\\Users\\mrkf9\\Documents\\KozakAron\\python\\ammonia_TDK_simulation\\Bubble_dynamics_simulation-ammonia\\full_bubble_model.py'>

In [2]:
"""Base settings"""

directory = 'test 1'
file_base_name = 'output_'

In [3]:
"""Load all CSV files from directory into a dataframe (all_data)"""

# list all files in a given directory
def list_all_files(directory):
    list_of_files = os.listdir(directory)
    all_files = []
    # Iterate over all the entries
    for entry in list_of_files:
        # Create full path
        full_path = os.path.join(directory, entry)
        # If entry is a directory then get the list of files in this directory 
        if os.path.isdir(full_path):
            all_files = all_files + list_all_files(full_path)
        else:
            all_files.append(full_path)
                
    return all_files

# create a dataframe
all_data = pd.DataFrame()

# Load all files
parent_dir = os.getcwd()
save_dir = os.path.join(parent_dir, directory)
all_files = list_all_files(save_dir)
num = 0
print(f'Found files:')
for file in all_files:
    # check if it's a CSV starting with file_base_name
    file_name = os.path.basename(file)
    if file_name[-4:] != '.csv':
        continue
    if file_name[:len(file_base_name)] != file_base_name:
        continue
    
    # Load csv into all_data
    current_data = pd.read_csv(file)
    print(f'\t{file_name}\t({current_data.shape[0]} rows)')
    all_data = pd.concat([all_data, current_data])
    num += 1
    
# Print some stats:
print(f'_______________________________________')
print(f'total number of files: {num}')
total = all_data.shape[0]
print(f'total:  {total} rows   (100 %)')
for error_code in range(7):
    num = all_data.loc[(all_data['error_code'] == error_code)].shape[0]
    print(f'error code {error_code}: {num} rows   ({(100*num/total):.2f} %)')
num = all_data.loc[(all_data['T_max'] > 6000.0)].shape[0]
print(f'too hot: {num} rows    ({100*num/total:.2f} %)')
print(f'_______________________________________')

Found files:
	output_1-checkpoint.csv	(3116 rows)
	output_2-checkpoint.csv	(1 rows)
	output_3-checkpoint.csv	(9 rows)
	output_5-checkpoint.csv	(9 rows)
	output_6-checkpoint.csv	(9 rows)
	output_1.csv	(9 rows)
	output_2.csv	(9 rows)
	output_3.csv	(9 rows)
	output_4.csv	(9 rows)
	output_5.csv	(9 rows)
	output_6.csv	(9 rows)
_______________________________________
total number of files: 11
total:  3198 rows   (100 %)
error code 0: 3005 rows   (93.96 %)
error code 1: 17 rows   (0.53 %)
error code 2: 29 rows   (0.91 %)
error code 3: 10 rows   (0.31 %)
error code 4: 137 rows   (4.28 %)
error code 5: 0 rows   (0.00 %)
error code 6: 0 rows   (0.00 %)
too hot: 862 rows    (26.95 %)
_______________________________________


In [9]:
"""Locate valid (good_data) and wrong (wrong_data) simulations"""

# Get valid datas
good_data = all_data.loc[
    (all_data['error_code'] < 4) &
    (all_data['energy_efficiency'] > 0.0) & (all_data['energy_efficiency'] == all_data['energy_efficiency']) # positive not NaN
    # or set negative energy to inf: good_data.loc[good_data['energy'] < 0.0, 'energy'] = 1e10
]

# Everything that's not in good_data
wrong_data = pd.concat([good_data, all_data]).drop_duplicates(keep=False)

# Sort by energy
good_data = good_data.sort_values(['energy_efficiency'], ascending=True)

# New column
good_data['logE'] = np.log10(good_data['energy_efficiency'])
print(f'Good data: {good_data.shape[0]} rows ({100*good_data.shape[0]/all_data.shape[0]: .2f} %)')

Good data: 2583 rows ( 80.77 %)


In [19]:
"""Let's see the dataframe"""

good_data[['ID', 'R_E', 'ratio', 'P_amb', 'alfa_M', 'T_inf', 'P_v', 'mu_L', 'gases', 'fractions', 'surfactant', 'freq1', 'freq2', 'pA1', 'pA2', 'theta_phase', 'error_code', 'elapsed_time', 'steps', 'collapse_time', 'T_max', 'n_H2', 'n_O2', 'n_NH3', 'expansion_work', 'dissipated_acoustic_energy', 'energy_efficiency']]

,ID,R_E,ratio,P_amb,alfa_M,T_inf,P_v,mu_L,gases,fractions,...,elapsed_time,steps,collapse_time,T_max,n_H2,n_O2,n_NH3,expansion_work,dissipated_acoustic_energy,energy_efficiency
2284,2289,0.000245,5.5,5066.25,0.35,293.15,2338.339978,0.001018,2 31,0.65 0.35,...,2.446501,3525,5.443416e-04,4980.659719,7.312199e-11,0.0,1.316607e-11,5.141784e-05,0.0,2.293123e+02
2243,2251,0.000240,5.5,5066.25,0.35,293.15,2338.339978,0.001018,2 31,0.65 0.35,...,2.621323,3686,5.331145e-04,4966.948959,6.892565e-11,0.0,1.237575e-11,4.836109e-05,0.0,2.294533e+02
2320,2327,0.000250,5.5,5066.25,0.35,293.15,2338.339978,0.001018,2 31,0.65 0.35,...,2.565380,3722,5.555688e-04,4994.014022,7.752589e-11,0.0,1.396204e-11,5.460075e-05,0.0,2.296251e+02
2206,2213,0.000235,5.5,5066.25,0.35,293.15,2338.339978,0.001018,2 31,0.65 0.35,...,2.728213,3762,5.218875e-04,4952.862536,6.493197e-11,0.0,1.159151e-11,4.542794e-05,0.0,2.301191e+02
2358,2365,0.000255,5.5,5066.25,0.35,293.15,2338.339978,0.001018,2 31,0.65 0.35,...,2.728213,3742,5.667961e-04,5007.026483,8.214165e-11,0.0,1.476363e-11,5.791236e-05,0.0,2.303286e+02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72,78,0.000003,2.0,5066.25,0.35,293.15,2338.339978,0.001018,2 31,0.65 0.35,...,3.381547,5875,1.246561e-06,318.495694,1.599914e-15,0.0,3.927750e-137,1.595432e-11,0.0,2.385088e+121
35,40,0.000002,2.0,5066.25,0.35,293.15,2338.339978,0.001018,2 31,0.65 0.35,...,2.873066,4745,7.094833e-07,315.469953,6.884355e-16,0.0,7.302427e-139,6.673124e-12,0.0,5.365766e+122
0,2,0.000001,2.0,5066.25,0.35,293.15,2338.339978,0.001018,2 31,0.65 0.35,...,2.187765,3512,2.668599e-07,311.401137,1.664495e-16,0.0,4.213674e-141,1.563860e-12,0.0,2.179248e+124
32,39,0.000002,1.5,5066.25,0.35,293.15,2338.339978,0.001018,2 31,0.65 0.35,...,2.190762,3927,5.770351e-07,296.545030,6.884353e-16,0.0,1.053335e-146,1.785080e-12,0.0,9.950865e+129


In [20]:
"""Print some statistics"""

good_data[['ID', 'R_E', 'ratio', 'P_amb', 'alfa_M', 'T_inf', 'P_v', 'mu_L', 'gases', 'fractions', 'surfactant', 'freq1', 'freq2', 'pA1', 'pA2', 'theta_phase', 'error_code', 'elapsed_time', 'steps', 'collapse_time', 'T_max', 'n_H2', 'n_O2', 'n_NH3', 'expansion_work', 'dissipated_acoustic_energy', 'energy_efficiency']].describe()

,ID,R_E,ratio,P_amb,alfa_M,T_inf,P_v,mu_L,surfactant,freq1,...,elapsed_time,steps,collapse_time,T_max,n_H2,n_O2,n_NH3,expansion_work,dissipated_acoustic_energy,energy_efficiency
count,2583.000000,2583.000000,2583.000000,2583.00,2.583000e+03,2.583000e+03,2.583000e+03,2.583000e+03,2583.0,2583.0,...,2583.000000,2583.000000,2583.000000,2583.000000,2.583000e+03,2.583000e+03,2.583000e+03,2.583000e+03,2583.0,2.583000e+03
mean,1494.080527,0.000145,10.607820,5066.25,3.500000e-01,2.931500e+02,2.338340e+03,1.017649e-03,1.0,20000.0,...,9.979867,8913.329075,0.000277,4217.526461,5.185559e-11,-4.615374e-43,2.034461e-12,3.881248e-04,0.0,1.683309e+127
std,902.300847,0.000111,5.174482,0.00,1.521300e-14,1.381563e-11,3.638683e-11,8.241532e-18,0.0,0.0,...,31.677835,7109.543317,0.000292,2760.501455,7.089215e-11,1.914953e-41,4.737913e-12,9.560832e-04,0.0,6.880864e+128
min,1.000000,0.000001,1.500000,5066.25,3.500000e-01,2.931500e+02,2.338340e+03,1.017649e-03,1.0,20000.0,...,0.056942,95.000000,0.000000,293.160639,1.663573e-16,-9.591048e-40,7.384738e-148,4.216827e-13,0.0,2.293123e+02
25%,710.500000,0.000035,6.000000,5066.25,3.500000e-01,2.931500e+02,2.338340e+03,1.017649e-03,1.0,20000.0,...,2.565380,3518.000000,0.000012,1169.854701,3.068266e-13,0.000000e+00,1.034058e-40,5.005599e-07,0.0,1.115320e+03
50%,1450.000000,0.000135,10.500000,5066.25,3.500000e-01,2.931500e+02,2.338340e+03,1.017649e-03,1.0,20000.0,...,5.545337,7028.000000,0.000177,4663.189527,1.356573e-11,0.000000e+00,1.000029e-14,2.233349e-05,0.0,5.204060e+03
75%,2295.500000,0.000245,15.000000,5066.25,3.500000e-01,2.931500e+02,2.338340e+03,1.017649e-03,1.0,20000.0,...,11.283476,15092.000000,0.000484,6562.187178,8.694089e-11,0.000000e+00,1.416288e-12,1.998146e-04,0.0,2.775454e+29
max,3115.000000,0.000350,20.000000,5066.25,3.500000e-01,2.931500e+02,2.338340e+03,1.017649e-03,1.0,20000.0,...,326.504529,23690.000000,0.000999,9127.985123,2.623404e-10,1.429857e-41,3.195160e-11,7.005043e-03,0.0,3.352899e+130


In [31]:
"""Plot a certain simulation"""

# converts a line of all_data to a dictionary containing the control parameters (cpar)
def line_to_dict(line):
    gases = str(line['gases'])
    fractions = str(line['fractions'])
    return de.dotdict(dict(
        ID=line['ID'],
        R_E=line['R_E'], # [m]
        ratio=line['ratio'], # [-]
        P_amb=line['P_amb'], # [Pa]
        alfa_M=line['alfa_M'], # [-]
        T_inf=line['T_inf'], # [m]
        surfactant=line['surfactant'], # [-]
        P_v=line['P_v'], # [Pa]
        mu_L=line['mu_L'], # [Pa*s]
        gases=[int(index) for index in gases.split(' ') if index!=''],  # indexes of species in initial bubble
        fractions=[float(frac) for frac in fractions.split(' ') if frac!=''], # molar fractions of species in initial bubble
    ))
cpar = line_to_dict(good_data.iloc[1])   # choose the most energy efficient one
de.plot(cpar)

LSODA solver failed, Radau solver had a fatal error (NO SOLUTION!)
